In [1]:
import datetime, time
import simpy

import shapely.geometry


import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot
import openclsim.plugins as plugins

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}
keep_resources = {}

In [2]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)

location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)


In [3]:
from_site = Site(
    env=my_env,
    name="Winlocatie",
    geometry=location_from_site,
    capacity=12,
    level=12,
)
to_site = Site(
    env=my_env,
    name="Dumplocatie",
    geometry=location_to_site,
    capacity=12,
    level=0,
)

In [4]:
hopper = TransportProcessingResource(
    env=my_env,
    name="Hopper 01",
    geometry=location_from_site,
    loading_rate=1,
    unloading_rate=1,
    capacity=4,
    compute_v=lambda x: 10,
)

## Definition of the activities

In [5]:
DelaySequenceActivity =  type(
    "TestShiftActivity",
    (
        plugins.HasDelayPlugin,
        model.SequentialActivity,  # the order is critical!
    ),
    {},
)

DelayWhileActivity =  type(
    "TestShiftActivity",
    (
        plugins.HasDelayPlugin,
        model.WhileActivity,  # the order is critical!
    ),
    {},
)

DelayMoveActivity =  type(
    "TestMoveActivity",
    (
        plugins.HasDelayPlugin,
        model.MoveActivity,  # the order is critical!
    ),
    {},
)


DelayShiftActivity =  type(
    "TestShiftActivity",
    (
        plugins.HasDelayPlugin,
        model.ShiftAmountActivity,  # the order is critical!
    ),
    {},
)

DelayBasicActivity =  type(
    "TestShiftActivity",
    (
        plugins.HasDelayPlugin,
        model.BasicActivity,  # the order is critical!
    ),
    {},
)

In [7]:
single_run = [
    DelayMoveActivity(
        env=my_env,
        name="sailing empty",
        registry=registry,
        mover=hopper,
        destination=from_site,
        delay_percentage=10,
    ),
    DelayShiftActivity(
        env=my_env,
        name="Transfer MP",
        registry=registry,
        processor=hopper,
        origin=from_site,
        destination=hopper,
        amount=4,
        duration=10,
        delay_percentage=10,
    ),
    DelayMoveActivity(
        env=my_env,
        name="sailing filler",
        registry=registry,
        mover=hopper,
        destination=to_site,
        delay_percentage=10,
    ),
    DelayShiftActivity(
        env=my_env,
        name="Transfer TP",
        registry=registry,
        processor=hopper,
        origin=hopper,
        destination=to_site,
        amount=4,
        duration=10,
        delay_percentage=10,
    ),
    DelayBasicActivity(
        env=my_env,
        name="Basic activity",
        registry=registry,
        duration=0,
        additional_logs=[hopper],
        delay_percentage=10,
    ),
]


activity = DelaySequenceActivity(
    env=my_env,
    name="Single run process",
    registry=registry,
    sub_processes=single_run,
    delay_percentage=10,
)

while_activity = DelayWhileActivity(
    env=my_env,
    name="while",
    registry=registry,
    sub_processes=[activity],
    condition_event=[{"type": "container", "concept": to_site, "state": "full"}],
    delay_percentage=10,
)


In [8]:
model.register_processes([while_activity])
my_env.run()

In [9]:
pd.concat([plot.get_log_dataframe(act, [while_activity, activity, *single_run]) for act in single_run]).sort_values(by='Timestamp')

,Activity,Timestamp,ActivityState,type,ref
0,sailing empty,1970-01-01 00:00:00.000000,START,NaN,NaN
1,sailing empty,1970-01-01 00:00:00.000000,STOP,NaN,NaN
2,sailing empty,1970-01-01 00:00:00.000000,WAIT_START,plugin,delay
3,sailing empty,1970-01-01 00:00:00.000000,WAIT_STOP,plugin,delay
0,Transfer MP,1970-01-01 00:00:00.000000,START,NaN,NaN
1,Transfer MP,1970-01-01 00:00:10.000000,STOP,NaN,NaN
2,Transfer MP,1970-01-01 00:00:10.000000,WAIT_START,plugin,delay
3,Transfer MP,1970-01-01 00:00:11.000000,WAIT_STOP,plugin,delay
0,sailing filler,1970-01-01 00:00:11.000000,START,NaN,NaN
2,sailing filler,1970-01-01 00:15:53.824591,WAIT_START,plugin,delay


In [10]:
objects = [hopper, from_site, to_site, while_activity, activity]
objects.extend(single_run)
plot.vessel_planning(objects)